# Study of the narrative
Study of the narrative through sets of hands presented together within a painting and its corresponding iconography
- Get significant KMeans clusters
- Group hands from same painting through their cluster representations
- Compare patterns within same iconographies


## Imports

In [1]:
import os
import cv2
import threading
import time
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import sys
import imageio
import random
import sqlite3
import itertools
import json
import shutil
import ntpath
import math
import pandas as pd

from time import sleep
from datetime import datetime
from ipynb.fs.full.write_html import write_html, write_js
from random import randint
from random import randrange

import KPmanager

In [2]:
with open('cluster_labels_kmeans_pca_all.pkl','rb') as f:
    labels_kmeans_pca = pickle.load(f)

In [3]:
with open('cluster_labels_spectral_all.pkl','rb') as f:
    labels_spectral =  pickle.load(f)

In [4]:
with open('all_hands_images.pkl', 'rb') as handle:
    files_name = pickle.load(handle)

In [5]:
with open('all_hands_features_JOINT_UNIT.pkl', 'rb') as handle:
    hands_features = pickle.load(handle)

In [6]:
with open('iconclass_dic.pkl', 'rb') as handle:
    iconclass_dic = pickle.load(handle)

In [7]:
iconclass_dic

{'11H': 'saints',
 '71P': 'the story of the prophet Daniel; his visions and prophecies',
 '11I': 'prophets, sibyls, evangelists, Doctors of the Church; persons ~ the Bible (not in biblical context)',
 '71O': 'prophets (before and during the Babylonian Captivity)',
 '11B': "the Holy Trinity, 'Trinitas coelestis'; Father, Son and Holy Ghost ~ Christian religion",
 '43A': 'festivities',
 '73F': 'lives and acts of the apostles of Christ; epistles',
 '71S': 'prophets (after the Babylonian Captivity)',
 '71C': 'Genesis: the patriarchs',
 '33A': 'non-aggressive relationships',
 '71D': 'Genesis: the story of Joseph',
 '48C': 'the arts; artists',
 '71A': 'Genesis from the creation to the expulsion from paradise, and later years of Adam and Eve',
 '11E': 'the Holy Ghost',
 '73C': 'public life of Christ: from his baptism until the Passion',
 '73B': 'birth and youth of Christ',
 '73E': "events from Resurrection to Pentecost (Matthew 28; Mark 16; Luke 24; John 20-21; Acts 1:3-11); Mary's and Joseph

In [8]:
DB_df_ = pd.read_pickle("./DB_df_keywords_DE.pkl")

In [9]:
DB_df_.head(3)

,index,object_ID,title,artist_GND,artist_name,date_begin,date_end,current_location,type,medium,dimensions,img_digital,century,id,keywords
54,54,8055384,Die Heiligen Scholastika und Benedikt empfange...,ulan500017301 & gnd123097843,Paolo de Matteis,1685.0,1725.0,Cava de' Tirreni,Gemälde,Öl,225 x 180 cm,bhim00035651,1650.0,bhim00035651_2_hand_right.jpg,"[[(11H, 1450), (71P, 119), (11I, 71), (71O, 41..."
87,87,8028831,Madonna del Soccorso,unknown,Niccolò di Liberatore di Giacomo di Mariano,1497.0,1497.0,unknown,Gemälde,Tempera & Leinwand,150 x 107 cm,bhim00017285,1450.0,bhim00017285_0_hand_left.jpg,"[[(11F, 131), (11H, 7), (11I, 1), (33A, 1)]]"
93,93,8088949,Bauern an der Tränke,ulan500007713 & gnd136667031,Michelangelo Cerquozzi,1620.0,1660.0,unknown,Gemälde,Öl,"51 x 44,5 cm",bhim00036067,1600.0,bhim00036067_1_hand_left.jpg,"[[(47I, 115), (47G, 70), (11H, 41), (71I, 37),..."


In [10]:
iconography_key = []
iconography_name = []

for index, row in DB_df_.iterrows():
    if len(row['keywords'][0])>0:
        key = row['keywords'][0][0][0]

        iconography_key.append(key)
        iconography_name.append(iconclass_dic[key])
    else:
        iconography_key.append('None')
        iconography_name.append('None')

In [11]:
DB_df_['iconography_main'] = iconography_key
DB_df_['iconography'] = iconography_name

In [12]:
print(files_name[0])
print(labels_kmeans_pca[0])
print(labels_spectral[0])

bhim00037788_0_hand_right.jpg
6
18


In [13]:
cluster_pairs = list(zip(files_name, labels_spectral))

In [14]:
cluster_pairs[0]

('bhim00037788_0_hand_right.jpg', 18)

In [15]:
hands_combination = {}
for file_name, label in zip(files_name, labels_spectral):
    img_digital = DB_df_.loc[DB_df_['id'] == file_name, 'img_digital'].values[0]
    if img_digital in hands_combination:
        hands_combination[img_digital].append(label)
        continue
    hands_combination[img_digital] = [label]

In [22]:
hands_combination

{'bhim00037788': [18, 47],
 'bh012186': [3],
 'bh013263': [6, 21],
 'bhim00037212': [28, 34, 2, 36, 0, 10, 45],
 'bh012634': [22],
 'bhim00024287': [3],
 'bhim00017401': [19],
 'bh013929': [24],
 'bhim00017456': [25],
 'bh007998': [47],
 'bh052628': [9, 25],
 'bhim00015955': [13],
 'bhim00033989': [31],
 'bh000775': [4, 36, 14],
 'bh012303': [19],
 'bh002682': [37],
 'bh013546': [29, 17, 45],
 'bh013200': [19, 24, 36, 44],
 'bh013091': [7, 0, 37, 45, 37],
 'bhim00037058': [6, 19, 45, 14, 15, 21, 31],
 'bh012280': [18, 32],
 'bh001533': [6],
 'bh002586': [5, 39, 26, 32, 25, 3],
 'bh013231': [44, 4, 47],
 'bhim00017293': [22],
 'bhim00038638': [36],
 'bh013276': [6],
 'bhim00024435': [3],
 'bh209938': [18, 47, 34],
 'bhpd68259': [3, 37],
 'bhim00024342': [17],
 'bh001993': [19],
 'bhim00027728': [26, 1, 14],
 'bh013281': [19, 26],
 'bhim00017324': [43, 45, 36, 19],
 'bhim00037194': [25],
 'bhim00037841': [22, 15, 11, 44, 5, 5],
 'bh012257': [7],
 'bh000889': [12, 14],
 'bhim00038652': [3

In [29]:
for key, item in hands_combination.items():
    hands_combination[key] = sorted(item)

In [30]:
hands_combination

{'bhim00037788': [18, 47],
 'bh012186': [3],
 'bh013263': [6, 21],
 'bhim00037212': [0, 2, 10, 28, 34, 36, 45],
 'bh012634': [22],
 'bhim00024287': [3],
 'bhim00017401': [19],
 'bh013929': [24],
 'bhim00017456': [25],
 'bh007998': [47],
 'bh052628': [9, 25],
 'bhim00015955': [13],
 'bhim00033989': [31],
 'bh000775': [4, 14, 36],
 'bh012303': [19],
 'bh002682': [37],
 'bh013546': [17, 29, 45],
 'bh013200': [19, 24, 36, 44],
 'bh013091': [0, 7, 37, 37, 45],
 'bhim00037058': [6, 14, 15, 19, 21, 31, 45],
 'bh012280': [18, 32],
 'bh001533': [6],
 'bh002586': [3, 5, 25, 26, 32, 39],
 'bh013231': [4, 44, 47],
 'bhim00017293': [22],
 'bhim00038638': [36],
 'bh013276': [6],
 'bhim00024435': [3],
 'bh209938': [18, 34, 47],
 'bhpd68259': [3, 37],
 'bhim00024342': [17],
 'bh001993': [19],
 'bhim00027728': [1, 14, 26],
 'bh013281': [19, 26],
 'bhim00017324': [19, 36, 43, 45],
 'bhim00037194': [25],
 'bhim00037841': [5, 5, 11, 15, 22, 44],
 'bh012257': [7],
 'bh000889': [12, 14],
 'bhim00038652': [3

In [160]:
from_ = []
to_ = []
icon_ = []
#century_ = []
image_ = []
for key, combination in hands_combination.items():
    pairs = [(a, b) for idx, a in enumerate(combination) for b in combination[idx + 1:]]
    for (a, b) in pairs:
        from_.append(a)
        to_.append(b)
        icon_.append(DB_df_.loc[DB_df_['img_digital'] == key, 'iconography'].values[0])
        image_.append(key)
        #century_.append(DB_df_.loc[DB_df_['img_digital'] == key, 'century'].values[0])

In [161]:
#century_[3]
image_[0]

'bhim00037788'

In [162]:
#edge_df_ = pd.DataFrame(list(zip(from_, to_, icon_, century_)),columns=['source', 'target', 'legend', 'century']) #source #target

edge_df_ = pd.DataFrame(list(zip(from_, to_, icon_, image_)),columns=['source', 'target', 'legend', 'image']) #source #target

In [96]:
print(len(edge_df_))
edge_df_.head(5)

7335


,source,target,legend,image
0,18,47,saints,bhim00037788
1,6,21,Christ,bh013263
2,0,2,betrothal and marriage,bhim00037212
3,0,10,betrothal and marriage,bhim00037212
4,0,28,betrothal and marriage,bhim00037212


In [95]:
edge_df_.loc[edge_df_['legend']=='betrothal and marriage']

,source,target,legend,image
2,0,2,betrothal and marriage,bhim00037212
3,0,10,betrothal and marriage,bhim00037212
4,0,28,betrothal and marriage,bhim00037212
5,0,34,betrothal and marriage,bhim00037212
6,0,36,betrothal and marriage,bhim00037212
7,0,45,betrothal and marriage,bhim00037212
8,2,10,betrothal and marriage,bhim00037212
9,2,28,betrothal and marriage,bhim00037212
10,2,34,betrothal and marriage,bhim00037212
11,2,36,betrothal and marriage,bhim00037212


In [163]:
edge_df_ = edge_df_.drop_duplicates()

In [164]:
edge_df_ = edge_df_.groupby(['source', 'target', 'legend']).image.apply(list).reset_index()

In [165]:
edge_df_

,source,target,legend,image
0,0,0,gods of the waters: the older dynasty (first s...,[bh102378]
1,0,0,lesser divinities of Heaven ~ serving and atte...,[bhim00036998]
2,0,0,saints,"[bhim00012232, bhim00034078, bhim00033567]"
3,0,1,gods of the earth and fertility ~ life in moun...,[bhim00038611]
4,0,1,saints,[bhim00009148]
...,...,...,...,...
3749,47,48,saints,[bh000922]
3750,47,49,male persons from classical history,[bh013346]
3751,48,48,saints,[bhim00027604]
3752,48,49,lives and acts of the apostles of Christ; epis...,[bhim00034316]


In [114]:
edge_df.loc[edge_df['legend']=='Judgment of Conduct']

,source,target,legend,image,weight
1455,8,12,Judgment of Conduct,['bh002717'],1
1732,11,11,Judgment of Conduct,['bhim00015972'],1
1784,11,26,Judgment of Conduct,['bhim00015972'],1
1788,11,27,Judgment of Conduct,['bhim00015972'],1
1800,11,30,Judgment of Conduct,['bhim00015972'],1
1861,11,47,Judgment of Conduct,['bhim00015972'],1
2993,26,27,Judgment of Conduct,"['bh290776', 'bhim00015972']",2
3003,26,30,Judgment of Conduct,['bhim00015972'],1
3058,26,47,Judgment of Conduct,"['bh290776', 'bhim00015972']",2
3069,27,30,Judgment of Conduct,['bhim00015972'],1


In [166]:
count_icon = []
count_edge = []
#centuries = []
for key, row in edge_df_.iterrows():
    #dup = edge_df_.loc[((edge_df_['source']==row['source']) | (edge_df_['source']==row['target'])) 
     #                  & ((edge_df_['target']==row['target']) | (edge_df_['target']==row['source'])) 
      #                 & (edge_df_['legend']==row['legend'])]
    
    dup_edge = len(edge_df_.loc[(edge_df_['source']==row['source'])
                             & (edge_df_['target']==row['target'])])
    
    dup_icon = len(row['image'])
    
    #cent = edge_df_.loc[(edge_df_['source']==row['source']) & (edge_df_['target']==row['target']), 'century']
    count_icon.append(dup_icon)
    count_edge.append(dup_edge)

In [167]:
edge_df_['weight_icon'] = count_icon
edge_df_['weight_edge'] = count_edge

In [168]:
edge_df_

,source,target,legend,image,weight_icon,weight_edge
0,0,0,gods of the waters: the older dynasty (first s...,[bh102378],1,3
1,0,0,lesser divinities of Heaven ~ serving and atte...,[bhim00036998],1,3
2,0,0,saints,"[bhim00012232, bhim00034078, bhim00033567]",3,3
3,0,1,gods of the earth and fertility ~ life in moun...,[bhim00038611],1,2
4,0,1,saints,[bhim00009148],1,2
...,...,...,...,...,...,...
3749,47,48,saints,[bh000922],1,1
3750,47,49,male persons from classical history,[bh013346],1,1
3751,48,48,saints,[bhim00027604],1,1
3752,48,49,lives and acts of the apostles of Christ; epis...,[bhim00034316],1,2


In [169]:
len(edge_df_)

3754

In [32]:
#edge_df = edge_df_.loc[edge_df_['legend'] == 'birth and youth of Christ']

In [170]:
#edge_df_['century'] = edge_df_['century'].astype(str)
edge_df_['image'] = edge_df_['image'].astype(str)

In [62]:
#edge_df = edge_df_.drop_duplicates()
#edge_df = edge_df_.loc[edge_df_.astype(str).drop_duplicates().index]

In [171]:
len(edge_df)

3754

In [172]:
edge_df = edge_df_

In [117]:
nodes = [i for i in range(50)]
nodes_names = [str(i) for i in range(50)]

In [118]:
node_df = pd.DataFrame(list(zip(nodes, nodes_names)), columns = ['id', 'name'])

In [119]:
edge_df.head(3)

,source,target,legend,image,weight
0,0,0,gods of the waters: the older dynasty (first s...,['bh102378'],1
1,0,0,lesser divinities of Heaven ~ serving and atte...,['bhim00036998'],1
2,0,0,saints,"['bhim00012232', 'bhim00034078', 'bhim00033567']",3


In [120]:
node_df.head(3)

,id,name
0,0,0
1,1,1
2,2,2


In [ ]:
filepath = 'cluster_network_vis/edge_df.csv'  
#edge_df.to_csv(filepath, index=False)
edge_df = pd.read_csv(filepath)

In [ ]:
filepath = 'cluster_network_vis/node_df.csv'  
#node_df.to_csv(filepath, index=False)
node_df = pd.read_csv(filepath)

In [29]:
edge_json = edge_df_.to_json(orient="records")
node_json = node_df.to_json(orient="records")

In [ ]:
json.dumps(json.JSONDecoder().decode(node_json))

In [ ]:
final_json = {"nodes": json.loads(node_json), "links": json.loads(edge_json)}

In [ ]:
final_json

In [ ]:
import json
with open('cluster_network_vis/data.json', 'w') as f:
    json.dump(final_json, f)
     
# Opening JSON file
f = open('cluster_network_vis/data.json')
final_json = json.load(f)

In [ ]:
import xml.etree.ElementTree as gfg 
from xml.dom import minidom
import os

In [ ]:
root = minidom.Document()
  
graph = root.createElement('graph') 
root.appendChild(graph)
  
nodes = root.createElement('nodes')
graph.appendChild(nodes)

for i, node in node_df.iterrows():
    newNode = root.createElement('node')
    newNode.setAttribute('id', str(i))
    newNode.setAttribute('label', 'cluster')
    nodes.appendChild(newNode)

edges = root.createElement('edges')
graph.appendChild(edges)

for i, edge in edge_df.iterrows():
    newEdge = root.createElement('edge')
    newEdge.setAttribute('source', edge['from'])
    newEdge.setAttribute('target', edge['to'])
    edges.appendChild(newEdge)

xml_str = root.toprettyxml(indent ="\t") 
  
save_path_file = "gfg.xml"
  
with open(save_path_file, "w") as f:
    f.write(xml_str) 

In [ ]:
save_path_file = "cluster_network_vis/gfg.xml"

xml_str = gfg.parse(save_path_file)

In [ ]:
def GenerateGEXF(fileName) :

    root = gfg.Element("graph")
      
    m1 = gfg.Element("nodes")
    root.append (m1)
      
    b1 = gfg.SubElement(m1, "brand")
    b1.text = "Redmi"
    b2 = gfg.SubElement(m1, "price")
    b2.text = "6999"
      
    m2 = gfg.Element("edges")
    root.append (m2)
      
    c1 = gfg.SubElement(m2, "brand")
    c1.text = "Samsung"
    c2 = gfg.SubElement(m2, "price")
    c2.text = "9999"
      
    tree = gfg.ElementTree(root)
      
    with open (fileName, "wb") as files :
        tree.write(files)

In [38]:
from jaal import Jaal

In [41]:
from jaal.datasets import load_got
Jaal(edge_df, node_df).plot() #vis_opts={'physics':{'stabilization':{'iterations': 100}}} define the convergence iteration of network

Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Sep/2022 12:00:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2022 12:00:13] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2022 12:00:13] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2022 12:00:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2022 12:00:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2022 12:00:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2022 12:00:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2022 12:00:13] "POST /_dash-update-component HTTP/1.1" 200 -


No trigger


127.0.0.1 - - [19/Sep/2022 12:00:18] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.js.map HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2022 12:00:18] "GET /_dash-component-suites/dash/html/dash_html_components.min.js.map HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2022 12:00:18] "GET /_dash-component-suites/dash/dash_table/bundle.js.map HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2022 12:00:18] "GET /_dash-component-suites/dash/dcc/dash_core_components.js.map HTTP/1.1" 200 -


In [188]:
edge_df_

,source,target,legend,century,weight
0,18,47,saints,1750.0,12
1,6,21,Christ,NaN,1
2,28,34,betrothal and marriage,1700.0,1
3,28,2,betrothal and marriage,1700.0,1
4,28,36,betrothal and marriage,1700.0,4
...,...,...,...,...,...
8335,31,3,"agriculture, cattle-breeding, horticulture, fl...",1600.0,1
8336,6,24,saints,1700.0,23
8337,3,35,birth and youth of Christ,1900.0,1
8338,3,5,birth and youth of Christ,1900.0,2


In [138]:
all_legend = edge_df['legend'].unique()

In [139]:
from random import randint

color = []
n = len(all_legend)

for i in range(n):
    color.append('#%06X' % randint(0, 0xFFFFFF))
print(color)

['#0C45E8', '#753DF0', '#A4B44B', '#DFD009', '#9FB29C', '#2E080E', '#8A3AE4', '#B5088F', '#53FF18', '#4965FB', '#3F8CC4', '#7B0311', '#C7BD1C', '#2FDC0B', '#DDAE1E', '#E1B78E', '#56D9D2', '#D8A3A7', '#C8C646', '#175E82', '#ED2B94', '#0E6D86', '#88ECB2', '#469696', '#D08987', '#0351E8', '#54DDFC', '#4CA12F', '#42ECA6', '#2BEDBE', '#2D068B', '#89B28E', '#A62337', '#0ED386', '#5F10F5', '#A08B40', '#6C8D5E', '#3F07D8', '#E92662', '#E63C4B', '#992302', '#F41F14', '#E33065', '#6D0FD0', '#BA663E', '#380414', '#A24CA2', '#C07C2E', '#E4D80E', '#138E34', '#0B4AD0', '#214D24', '#459028', '#2D6305', '#091AED', '#5A9445', '#84E06D', '#528133', '#2B27D7', '#F23FCE', '#F5B10E', '#D9F7CE', '#8916DF', '#63FB46', '#E28833', '#67CF1D', '#C7A560', '#DCFD7D', '#A698D0', '#F19BED', '#9C2056', '#157768', '#D06E5E', '#016664', '#357D2B', '#9AE4E7', '#89EBC4', '#8B8E69', '#6C0063']


In [247]:
import importlib
importlib.reload(pyvis)
importlib.reload(pyvis.network)

<module 'pyvis.network' from '/home/bernasconi/anaconda3/lib/python3.8/site-packages/pyvis/network.py'>

In [251]:
from pyvis.network import Network
import json


# function to add to JSON
def write_json(new_data, filename='data.json'):
    with open(filename,'r+') as file:
          # First we load existing data into a dict.
        file_data = json.load(file)
        # Join new_data with file_data inside emp_details
        file_data["edges"].append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, indent = 4)

def createNetwork(html_path, node_df__, edge_df__):
    net = Network(height='1050px', width='100%', neighborhood_highlight=True, bgcolor='#222222')#font_color='white')
    net.barnes_hut()
    count=0
    edges_ = '['
    for i, node in node_df__.iterrows():
        net.add_node(node['id'], shape='image', image="SPECTRAL_kmeans_clustering/cl_"+str(i)+".png") #label=node['name'],
    
    for i, edge in edge_df__.iterrows():
        from_ = edge['source']
        to_ = edge['target']
        value_ = edge['weight_icon']
        title_ = edge['legend']
        color_ = color[list(all_legend).index(edge['legend'])]
        image_ = edge['image']
        
        # Data to be written
        new_edge = '{"from":'+str(from_)+',"to":'+str(to_)+',"value":'+str(value_)+',"title":"'+title_+'","color":"'+str(color_)+'","image":"'+image_+'"},'
        edges_+=new_edge
        #write_json(new_edge)
        
        net.add_edge(from_, to_, value=edge['weight_edge'], title=title_, color=color_,opacity=0.5, image=image_)
    edges_+=']'


    with open('data.txt', 'w') as f:
        f.write(edges_)
    with open("data.json", "w") as write_file:
        json.dump(edges_, write_file)
    net.show(html_path)

In [253]:
createNetwork('nx.html', node_df, edge_df)

In [152]:
edge_df.loc[(edge_df['source']== 26) & (edge_df['target']==27)]

,source,target,legend,image,weight
2992,26,27,"Exodus, Leviticus, Numbers, Deuteronomy, Joshu...",['bhim00033974'],1
2993,26,27,Judgment of Conduct,"['bh290776', 'bhim00015972']",2
2994,26,27,Passion of Christ,"['bhex-si-2016-48-206', 'bh108420']",2
2995,26,27,festivities,['bh000884'],1
2996,26,27,gods of the earth and fertility ~ life in moun...,['bhim00038611'],1
2997,26,27,saints,"['bhpd51921', 'bhim00038686', 'bhim00034108']",3


In [125]:
edge_df_christ = edge_df.loc[edge_df['legend'] == 'birth and youth of Christ']

In [117]:
options = {
          "interaction":{   
               "selectConnectedEdges": True
}}

net.options=options

In [122]:
#net.show_buttons(filter_=['physics']) #'edges', 'nodes'])
net.show('nx.html')